In [4]:
import json
import datetime
import pickle

import requests
from os import getcwd

import rpy2

In [10]:
signature = 'LM9'
url = f"https://raw.githubusercontent.com/sgosline/proteomicsTumorDeconv/main/signature_matrices/{signature}.txt"
directory = getcwd()
filename = f'{directory}/{signature}.txt'
r = requests.get(url)

f = open(filename,'w')
f.write(r.content.decode('utf-8'))

61549

In [11]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [13]:
%%R -i filename
ref <- read.csv(filename, sep = "\t", row.names = 1)

cellTypeNames <- colnames(ref)

marker <- list()
for (i in seq(dim(ref)[2])) {
  s = cellTypeNames[i]
  tempMarkers <- rownames(ref[ref[,s] > quantile(ref[,s], prob=0.75),])
  markers <- c()
  tempTb = 2 * ref[,cellTypeNames[cellTypeNames != s]] - ref[,s]
  markers <- tempMarkers[tempMarkers %in% rownames(ref[rowSums(tempTb < 0) == (length(cellTypeNames) - 1), ])]
  marker[[i]] <- tempMarkers
  if (length(marker[[i]]) == 0) {
      warning(paste('This signature has no genes that meet the 0.75% quantile and foldchange cutoff: ' ))
  }
}

names(marker) <- cellTypeNames

list.gene <- data.frame(marker)

print(list.gene)

         B.cells      T4.cells      T8.cells            DC      Basophil
1        ABHD17B         AHDC1       ABHD17B         ACSL4       ABHD17B
2         ALOX15           AK6         ADAM8          AIF1         ACSL4
3           AQP9       ALOX5AP         AHDC1           AK4         ADAM8
4          BANK1          AQP9           AK6           AK6        ADGRE2
5         BRI3BP       BCL2L15        ALOX12       APBB1IP        ADGRE3
6        C8orf82           BPI       ALOX5AP          ARSB        ADGRG3
7           CAMP       C8orf82       APBB1IP          ATL1       ALOX5AP
8           CD19          CAMP          ATL1        BRI3BP          ALS2
9           CD22        CGREF1         ATP7A       C9orf72       APBB1IP
10          CD37        CHI3L1       BCL2L15           CA8          ARSB
11          CD40         CHIT1       C8orf82         CARD9          ATL1
12         CD79A          CPTP       CCDC88B       CCDC88A         ATP7A
13         CD79B        CSF2RB          CD37       